In [19]:
import torch
cuda = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())              # True
print(torch.cuda.get_device_name(0))          # NVIDIA GeForce GTX 1650
print(torch.rand(2, 2).to("cuda"))            # Should work without error


True
NVIDIA GeForce GTX 1650
tensor([[0.5723, 0.3311],
        [0.7244, 0.6947]], device='cuda:0')


We will use the BBC News Classification Dataset

In [20]:
import pandas as pd
url = "https://raw.githubusercontent.com/susanli2016/PyCon-Canada-2019-NLP-Tutorial/master/bbc-text.csv"
df  = pd.read_csv(url)
df = df.sample(n=1000, random_state=42)  # Sample 1000 rows for faster processing

In [21]:
df[df.isna().any(axis=1)]  # Check for any NaN values in the DataFrame

,category,text


There is not any NaN value

In [22]:
data = df.to_dict('records')  # Convert DataFrame to a list of dictionaries

In [23]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [24]:
from sentence_transformers import SentenceTransformer
encoder = SentenceTransformer('all-MiniLM-L6-v2', device=cuda)

In [25]:
# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

In [26]:
# Create collection to store wines
qdrant.recreate_collection(
    collection_name="bbc_news",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

C:\Users\user\AppData\Local\Temp\ipykernel_13016\1224430878.py:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [27]:
# vectorize!
qdrant.upload_points(
    collection_name="bbc_news",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["text"]).tolist(),
            payload=doc,
        ) for idx, doc in enumerate(data) # data is the variable holding all the bbc_news
    ]
)

e:\user\Documents\Repos\IntroductiontoGenerativeAI\.venv\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [28]:
user_prompt = "How did the stock market react to major banking news?"

In [29]:
# Search time 

hits = qdrant.search(
    collection_name="bbc_news",
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=3
)
for hit in hits:
  print(hit.payload, "score:", hit.score)

{'category': 'business', 'text': 'stock market eyes japan recovery japanese shares have ended the year at their highest level since 13 july amidst hopes of an economic recovery during 2005.  the nikkei index of leading shares gained 7.6% during the year to close at 11 488.76 points. in 2005 it  will rise toward 13 000   predicted morgan stanley equity strategist naoki kamiyama. the optimism in the financial markets contrast sharply with pessimism in the japanese business community. earlier this month  the quarterly tankan survey of japanese manufacturers found that business confidence had weakened for the first time since march 2003.  slower economic growth  rising oil prices  a stronger yen and weaker exports were blamed for the fall in confidence. despite this  traders expect strength in the global economy to benefit japan  which has been close to sliding into recession in recent months. structural reform within japan and an anticipated end to the banking sector s bad debt problems s

C:\Users\user\AppData\Local\Temp\ipykernel_13016\2403408260.py:3: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = qdrant.search(


In [30]:
# define a variable to hold the search results
search_results = [hit.payload for hit in hits]
search_results

[{'category': 'business',
  'text': 'stock market eyes japan recovery japanese shares have ended the year at their highest level since 13 july amidst hopes of an economic recovery during 2005.  the nikkei index of leading shares gained 7.6% during the year to close at 11 488.76 points. in 2005 it  will rise toward 13 000   predicted morgan stanley equity strategist naoki kamiyama. the optimism in the financial markets contrast sharply with pessimism in the japanese business community. earlier this month  the quarterly tankan survey of japanese manufacturers found that business confidence had weakened for the first time since march 2003.  slower economic growth  rising oil prices  a stronger yen and weaker exports were blamed for the fall in confidence. despite this  traders expect strength in the global economy to benefit japan  which has been close to sliding into recession in recent months. structural reform within japan and an anticipated end to the banking sector s bad debt problem

In [31]:
# Now time to connect to the local large language model
from openai import OpenAI
client = OpenAI(
    base_url="http://127.0.0.1:8080/v1", # "http://<Your api-server IP>:port"
    api_key = "sk-no-key-required"
)
completion = client.chat.completions.create(
    model="LLaMA_CPP",
    messages=[
        {"role": "system", "content": "You are chatbot, a news specialist. Your top priority is to help guide users to the most relevant news articles based on their queries."},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": str(search_results)}
    ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="The stock market reacted positively to major banking news in 2005. In Ireland, the Irish stock market reached an all-time high, with the ISEQ index closing up 23 points to 6661.89, fueled by strong growth in banking and financial stocks. Allied Irish Banks, Ireland's largest company by capitalization, touched a five-year peak, while Bank of Ireland shares rose to their highest level since August 2002.\n\nIn the US, the Federal Reserve raised interest rates for the fourth time in five months, to 2%, in light of mounting evidence that the US economy was regaining steam. The Fed's decision was widely anticipated and was seen as a positive move for the stock market. The Dow Jones Industrial Average closed flat, but financial analysts broadly welcomed the Fed's move.\n\nIn Japan, the Nikkei index of leading shares gained 7.6% for the year, closing at 11,488.76 points, amidst hopes of an economic recovery. The optimism in the financial markets contrasted sharpl